In [2]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
from config import spot_client_id, spot_secret_key
import pprint
import json
import pandas as pd
import requests
import numpy as np

In [4]:
spotify = spotipy.Spotify()
credentials = oauth2.SpotifyClientCredentials(
        client_id=spot_client_id,
        client_secret=spot_secret_key)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

In [5]:
LastFM_df = pd.read_csv("Resources/LastFM_data.csv", index_col=0)
Apple_df = pd.read_csv("Resources/appleAPI_top5_songs_per_country.csv")

In [6]:
#Merge/Clean Data Sets

Apple_df = Apple_df.rename(columns={
    "Name" : "Song Title",
    "Artist" : "Artist(s)"
})

merge_df = LastFM_df.append(Apple_df, sort=False)
merge_df.replace(np.nan, '', regex=True, inplace=True)
merge_df.sort_values(by=['Country','Source','Rank'], inplace=True)
merge_df.reset_index(inplace=True, drop=True)

In [7]:
spot_id = []
for index, row in merge_df.iterrows():
    try:
        res = spotify.search(row["Song Title"], type="track", limit=1)
        spot_id.append(res["tracks"]["items"][0]["id"])
    except:
        print(f"Couldn't find {row['Song Title']}")
        spot_id.append("Error")

Couldn't find Ne reviens pas (feat. Heuss L'enfoiré)


In [8]:
merge_df["Spotify_ID"] = spot_id
merge_df = merge_df[merge_df.Spotify_ID != 'Error']
merge_df.reset_index(inplace=True, drop=True)
merge_df

,Country,Artist(s),Song Title,Rank,Source,Album,Genre,Spotify_ID
0,Australia,Tones and I,Dance Monkey,,Apple,Dance Monkey - Single,Pop,2XU0oxnq2qxCpomAAuJY8K
1,Australia,Regard,Ride It,,Apple,Ride It - Single,Dance,2tnVG71enUj33Ic2nFN6kZ
2,Australia,Dua Lipa,Don't Start Now,,Apple,Don't Start Now - Single,Pop,6WrI0LAC5M1Rw2MnX2ZvEg
3,Australia,Tones and I,Never Seen the Rain,,Apple,The Kids Are Coming - EP,Alternative,0JQ5MbyriK6ruD3t6RZ7ix
4,Australia,Arizona Zervas,ROXANNE,,Apple,ROXANNE - Single,Hip-Hop/Rap,696DnlkuDOXcMAnKlTgXXK
...,...,...,...,...,...,...,...,...
94,US,The Weeknd,Can't Feel My Face,1,LastFM,Beauty Behind the Madness,,22VdIZQfgXJea34mQxlt81
95,US,Drake,Hotline Bling,2,LastFM,Views,,0wwPcA6wtMf6HUMpIRdeP7
96,US,Tame Impala,The Less I Know the Better,3,LastFM,Currents,,6K4t31amVTZDgR3sKmwUJJ
97,US,The Killers,Mr. Brightside,4,LastFM,Hot Fuss,,7oK9VyNzrYvRFo7nQEYkWN


In [9]:
danceability = []
energy = []
key = []
loudness = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []

for index, row in merge_df.iterrows():
    try:
        track_id = row['Spotify_ID']
        track_features = spotify.audio_features(tracks=track_id)
        danceability.append(track_features[0]["danceability"])
        energy.append(track_features[0]["energy"])
        key.append(track_features[0]["key"])
        loudness.append(track_features[0]["loudness"])
        speechiness.append(track_features[0]["speechiness"])
        acousticness.append(track_features[0]["acousticness"])
        instrumentalness.append(track_features[0]["instrumentalness"])
        liveness.append(track_features[0]["liveness"])
        valence.append(track_features[0]["valence"])
        tempo.append(track_features[0]["tempo"])
        duration_ms.append(track_features[0]["duration_ms"])
    except:
        print(f"Couldn't find details for {row['Song Title']}")

In [10]:
merge_df["danceability"] = danceability
merge_df["energy"] = energy
merge_df["key"] = key
merge_df["loudness"] = loudness
merge_df["speechiness"] = speechiness
merge_df["acousticness"] = acousticness
merge_df["instrumentalness"] = instrumentalness
merge_df["liveness"] = liveness
merge_df["valence"] = valence
merge_df["tempo"] = tempo
merge_df["duration_ms"] = duration_ms

In [11]:
merge_df

,Country,Artist(s),Song Title,Rank,Source,Album,Genre,Spotify_ID,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,Australia,Tones and I,Dance Monkey,,Apple,Dance Monkey - Single,Pop,2XU0oxnq2qxCpomAAuJY8K,0.824,0.588,6,-6.400,0.0924,0.69200,0.000104,0.1490,0.513,98.027,209438
1,Australia,Regard,Ride It,,Apple,Ride It - Single,Dance,2tnVG71enUj33Ic2nFN6kZ,0.882,0.780,0,-4.256,0.0939,0.15900,0.000052,0.0973,0.867,117.945,157606
2,Australia,Dua Lipa,Don't Start Now,,Apple,Don't Start Now - Single,Pop,6WrI0LAC5M1Rw2MnX2ZvEg,0.794,0.793,11,-4.521,0.0842,0.01250,0.000000,0.0952,0.677,123.941,183290
3,Australia,Tones and I,Never Seen the Rain,,Apple,The Kids Are Coming - EP,Alternative,0JQ5MbyriK6ruD3t6RZ7ix,0.672,0.464,6,-5.624,0.0370,0.43000,0.000000,0.2840,0.271,92.069,200742
4,Australia,Arizona Zervas,ROXANNE,,Apple,ROXANNE - Single,Hip-Hop/Rap,696DnlkuDOXcMAnKlTgXXK,0.621,0.601,6,-5.616,0.1480,0.05220,0.000000,0.4600,0.457,116.735,163636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,US,The Weeknd,Can't Feel My Face,1,LastFM,Beauty Behind the Madness,,22VdIZQfgXJea34mQxlt81,0.705,0.769,9,-5.526,0.0426,0.11200,0.000000,0.1050,0.590,107.939,213520
95,US,Drake,Hotline Bling,2,LastFM,Views,,0wwPcA6wtMf6HUMpIRdeP7,0.891,0.625,2,-7.861,0.0558,0.00261,0.000176,0.0504,0.548,134.967,267067
96,US,Tame Impala,The Less I Know the Better,3,LastFM,Currents,,6K4t31amVTZDgR3sKmwUJJ,0.640,0.740,4,-4.083,0.0284,0.01150,0.006780,0.1670,0.785,116.879,216320
97,US,The Killers,Mr. Brightside,4,LastFM,Hot Fuss,,7oK9VyNzrYvRFo7nQEYkWN,0.356,0.924,1,-3.740,0.0808,0.00101,0.000000,0.0953,0.232,148.017,222587
